# Stop visualization EDA

#### Stop lookup api

In [29]:
from dotenv import load_dotenv
import os
import requests


load_dotenv()


API_KEY = os.getenv("API_KEY")

url = f"https://api.resrobot.se/v2.1/location.name?input=Göteborg&format=json&accessId={API_KEY}"

response = requests.get(url)

response


<Response [200]>

In [30]:
data = response.json()

print(data.keys())

dict_keys(['stopLocationOrCoordLocation', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'requestId'])


In [31]:
# 'stopLocationOrCoordLocation'
stops_data = data.get("stopLocationOrCoordLocation", [])
stops_data[:5] 


[{'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'}, 'cls': '2'},
    {'icon': {'res': 'prod_gen'}, 'cls': '4'},
    {'icon': {'res': 'prod_gen'}, 'cls': '8'},
    {'icon': {'res': 'prod_gen'}, 'cls': '16'},
    {'icon': {'res': 'prod_gen'}, 'cls': '64'},
    {'icon': {'res': 'prod_gen'}, 'cls': '128'},
    {'icon': {'res': 'prod_gen'}, 'cls': '256'}],
   'timezoneOffset': 60,
   'id': 'A=1@O=GÖTEBORG@X=11973479@Y=57708895@U=1@L=740098001@B=1@p=1737347805@',
   'extId': '740098001',
   'name': 'GÖTEBORG',
   'lon': 11.973479,
   'lat': 57.708895,
   'weight': 15625,
   'products': 222,
   'meta': True,
   'minimumChangeDuration': 'PT26M'}},
 {'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'}, 'cls': '2'},
    {'icon': {'res': 'prod_gen'}, 'cls': '4'},
    {'icon': {'res': 'prod_gen'}, 'cls': '8'},
    {'icon': {'res': 'prod_gen'}, 'cls': '16'},
    {'icon': {'res': 'prod_gen'}, 'cls': '64'},
    {'icon': {'res': 'prod_gen'}, 'cls': '128'}],
   'timezoneOffse

In [32]:
from backend.connect_to_api import ResRobot  
import pandas as pd


resrobot = ResRobot()


location = "Göteborg"
stops = resrobot.access_id_from_location(location)


df_stops = pd.DataFrame(stops)


print(df_stops.head())

# verkar inte gå att få alla hållplatser inom visa koordinater med denna api:n får testa en annan. Denna api:n tar bara hållplatser baserat på namn input.


Name                                               extId
GÖTEBORG                                           740098001
Göteborg Centralstation                            740000002
Göteborg Sävenäs lokstation                        740016365
GÖTEBORG GAMLESTADEN                               740098526
Göteborg Korsvägen                                 740015578
Göteborg Kungsportsplatsen                         740016358
Göteborg Stenpiren                                 740072430
Göteborg Eketrägatan                               740025624
Göteborg Vårväderstorget                           740025707
Göteborg Axel Dahlströms torg                      740025608
Empty DataFrame
Columns: []
Index: []


# Nearby stop api


In [33]:

url = f"https://api.resrobot.se/v2.1/location.nearbystops?originCoordLat=57.708895&originCoordLong=11.973479&format=json&accessId={API_KEY}"

response = requests.get(url)

response

<Response [200]>

In [36]:
data = response.json()  
data.keys() 

dict_keys(['stopLocationOrCoordLocation', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'requestId'])

In [37]:
stops_data = data.get("stopLocationOrCoordLocation", [])

stops_data

''' Varje objekt innehåller en "stoplocation" med fält som name, lon och lat, products(typ av transport), dist(avstånd till punkten?!) och extId(unik identifierare)'''

[{'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'}, 'cls': '2'},
    {'icon': {'res': 'prod_gen'}, 'cls': '4'},
    {'icon': {'res': 'prod_gen'}, 'cls': '16'},
    {'icon': {'res': 'prod_gen'}, 'cls': '128'}],
   'timezoneOffset': 60,
   'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
   'extId': '740000002',
   'name': 'Göteborg Centralstation',
   'lon': 11.973479,
   'lat': 57.708895,
   'weight': 9815,
   'dist': 0,
   'products': 150,
   'minimumChangeDuration': 'PT15M'}},
 {'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'},
     'cls': '64'},
    {'icon': {'res': 'prod_gen'}, 'cls': '128'}],
   'timezoneOffset': 60,
   'id': 'A=1@O=Göteborg Centralst Drottningt@X=11973740@Y=57707898@U=1@L=740030117@',
   'extId': '740030117',
   'name': 'Göteborg Centralst Drottningt',
   'lon': 11.97374,
   'lat': 57.707898,
   'weight': 4608,
   'dist': 112,
   'products': 192,
   'minimumChangeDuration': 'PT5M'}},
 {'StopLocation': {'p